# Práctica Final: Clasificación de Documentos con Scikit-learn y MLflow

En esta práctica, utilizarás un conjunto de datos de Scikit-learn (podeís usar el mismo que en el notebook de Intro MLFlow) para entrenar un modelo de clasificación de documentos. El objetivo es construir un modelo capaz de clasificar automáticamente documentos en categorías predefinidas.

Pasos a seguir:

    Exploración de Datos: Analiza el conjunto de datos proporcionado para comprender su estructura y contenido.

    Preprocesamiento de Texto: Realiza tareas de preprocesamiento de texto, como tokenización y vectorización, para preparar los datos para el modelado.

    Entrenamiento del Modelo: Utiliza algoritmos de clasificación de Scikit-learn para entrenar un modelo con los datos preprocesados.

    Evaluación del Modelo: Evalúa el rendimiento del modelo utilizando métricas de evaluación estándar como precisión y recall.

    Registro de Métricas con MLflow: Utiliza MLflow para registrar métricas y hiperparámetros durante el entrenamiento, facilitando la gestión y comparación de experimentos.


Nota: Dado que no voy a poder tener acceso a vuestros logs de MLFlow añadirme las imagenes de la interfaz de MLFlow en el notebook!

In [ ]:
try:
  import fastapi
  import mlflow
  import pyngrok
except:
  !pip install fastapi pyngrok mlflow
  import fastapi
  import mlflow
  import pyngrok
from google.colab import userdata

In [2]:
mlflow.set_experiment('Classifiers Scikit')
#with mlflow.start_run(run_name='Despliegue de Algos'):
#  mlflow.log_metric('m1', 1.0)
#  mlflow.log_param('n_estimators', 30)

get_ipython().system_raw('mlflow ui --port 5000 &')

from pyngrok import ngrok

ngrok.kill()

NGROK_AUTH_TOKEN = userdata.get('ngrok_token')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

ngrok_tunnel = ngrok.connect(addr='5000', proto='http', bind_tls=True)
print('El tracking UI:', ngrok_tunnel.public_url)

2024/05/19 18:32:18 INFO mlflow.tracking.fluent: Experiment with name 'Classifiers Scikit' does not exist. Creating a new experiment.


El tracking UI: https://df33-34-125-202-187.ngrok-free.app


In [3]:
def mlflow_log_score(model_name, metric_name, metric_value):
  with mlflow.start_run(run_name=model_name):
    mlflow.log_metric(metric_name, metric_value)
  #mlflow.log_param('n_stimators', i)
  #mlflow.sklearn.log_model(model, 'clf-model')


Valoración de varios modelos de clasificación

In [4]:
# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


import subprocess
import mlflow
import time

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025, random_state=42),
    SVC(gamma=2, C=1, random_state=42),
    GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
    DecisionTreeClassifier(max_depth=5, random_state=42),
    RandomForestClassifier(
        max_depth=5, n_estimators=10, max_features=1, random_state=42
    ),
    MLPClassifier(alpha=1, max_iter=1000, random_state=42),
    AdaBoostClassifier(algorithm="SAMME", random_state=42),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]

#init MLFlow
#mlflow_ui_process = subprocess.Popen(['mlflow', 'ui', '--port', '5000'])
#time.sleep(5)

X, y = make_classification(
    n_features=2, n_redundant=0, n_informative=2, random_state=1, n_clusters_per_class=1
)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [
    #make_moons(noise=0.3, random_state=0),
    make_circles(noise=0.2, factor=0.5, random_state=1),
    #linearly_separable,
]

# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )

    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        clf = make_pipeline(StandardScaler(), clf)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        mlflow_log_score(name, 'Accuracy on test', score)
        mlflow_log_score(name, 'acc', score)

## Generar .py de funciones y main con al menos dos argumentos de entrada.

In [16]:
%%writefile calculator.py
import argparse
from cals import *

def get_args():
  parser = argparse.ArgumentParser(description='Argumentos:')
  parser.add_argument('--calc', type=int, help='Operación matemática')
  #parser.add_argument('--nom_run', type=str, help='Nombre del run de entrenamiento')
  parser.add_argument('--pars', nargs='+', type=int, help='Lista de parámetros(operadores)')
  parser.add_argument('--reps', type=int, help='Amppunt of times to repeat using params', default=1)
  return parser.parse_args()

def main():
  args = get_args()
  reps = args.reps
  params = args.pars
  if len(params) < reps*2:
    print('Error. Debe dar suficientes parámetros para realizar las operaciones. Los parámetros deben ser al menos el doble que las operaciones')
    return
  for i in range(reps):
    params = args.pars[2*i:2*i+2]
    match args.calc:
      case 1:
        print(suma(*params))
      case 2:
        print(resta(*params))
      case 3:
        print(prod(*params))
      case 4:
        print(divd(*params))
      case 5:
        print(pow(*params))

if __name__ == '__main__':
  main()

Overwriting calculator.py


In [8]:
%%writefile cals.py
def suma(a, b):
  return a+b

def resta(a, b):
  return a-b

def prod(a, b):
  return a*b

def divd(a, b):
  return a/b

def pow(a, b):
  return a**b

Writing cals.py


In [52]:
!python calculator.py \
--calc 1 \
--pars 3 4 2 5 5  \
--reps 3

Error. You must give enough parameters to make operations. 2x params as reps at least


## Práctica parte FastAPI

### Para esta parte de la práctica teneis que generar un script con al menos 5 modulos app.get y dos de ellos tienen que ser pipelines de HF.

### Parte de la practica se tendra que entregar en capturas de pantalla. Las capturas de pantalla a adjuntas son las siguientes.

### 1. Captura de la pantalla docs con al menos 5 modulos.
### 2. Captura de cada una de los modulos con la respuesta dentro de docs.
### 3. Captura de cada uno de los modulos en la llamada https.
### 4. Todo el codigo usado durante el proceso. Notebooks y scripts.

### Despliegue

In [49]:
%%writefile hf.py
from fastapi import FastAPI
from transformers import pipeline

tags_metadata = [
    {
        "name": "Sentiment Analysis",
        "description": "Este grupo es de analisis de sentimiento",
    },
    {
        "name": "Q&A",
        "description": "Este grupo es de Question Answering",
    },
    {
        "name": "Summarization",
        "description": "Este grupo es de resumir",
    },
    {
        "name": "items",
        "description": "Manage items. So _fancy_ they have their own docs.",
        "externalDocs": {
            "description": "Items external docs",
            "url": "https://fastapi.tiangolo.com/",
        },
    },
]

app = FastAPI(openapi_tags=tags_metadata)
app._ctx = ''

@app.get('/sa', tags=['Sentiment Analysis'])
def sa(prompt):
  sa = pipeline('sentiment-analysis')
  return sa(prompt)

"""
Fija un contexto para el Q&A
"""
@app.get('/set_ctx', tags=['Q&A'])
def set_context(ctx):
  app._ctx = ctx
  return f'El contexto ha sido fijado: {app._ctx}'

@app.get('/get_ctx', tags=['Q&A'])
def get_ctx():
  return app._ctx

@app.get('/qa', tags=['Q&A'])
def qa(prompt):
  if app._ctx == '':
    return 'Debe fijar un contexto primero usando set_ctx'
  qa = pipeline('question-answering', model='BSC-LT/roberta-base-bne')
  return qa(prompt, app._ctx)

app._max_len = 512
@app.get('/set_max_length', tags=['Summarization'])
def set_max_len(length):
  app._max_len = lenght
  return f'Max length set to: {_max_length}'


@app.get('/sum', tags=['Summarization'])
def sum(text):
  import torch
  from transformers import BertTokenizerFast, EncoderDecoderModel
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  ckpt = 'mrm8488/bert2bert_shared-spanish-finetuned-summarization'
  tokenizer = BertTokenizerFast.from_pretrained(ckpt)
  model = EncoderDecoderModel.from_pretrained(ckpt).to(device)

  inputs = tokenizer([text], padding="max_length", truncation=True, max_length=app._max_len, return_tensors="pt")
  input_ids = inputs.input_ids.to(device)
  attention_mask = inputs.attention_mask.to(device)
  output = model.generate(input_ids, attention_mask=attention_mask)
  return tokenizer.decode(output[0], skip_special_tokens=True)
  generate_summary(text)

#@app.get('/trad/{en}')
#def trad(en):
#  from transformers import T5ForConditionalGeneration, T5Tokenizer
#
#  tr = pipeline('translation_EN_to_ES', model='jbochi/madlad400-3b-mt')
#  return tr(en)


Overwriting hf.py


In [50]:
import nest_asyncio
from pyngrok import ngrok, conf
NGROK_TOKEN = userdata.get('ngrok_token')
conf.get_default().auth_token = NGROK_TOKEN #cada uno os lo debéis generar en ngrok.

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url+'/docs')
nest_asyncio.apply()

Public URL: https://1b5c-34-125-202-187.ngrok-free.app/docs


In [ ]:
!uvicorn hf:app --port 8000